In [1]:
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1C/blp25_hatespeech_subtask_1C_train.tsv
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1C/blp25_hatespeech_subtask_1C_dev.tsv
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1C/blp25_hatespeech_subtask_1C_test.tsv

--2025-09-12 19:26:37--  https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1C/blp25_hatespeech_subtask_1C_train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8686966 (8.3M) [application/octet-stream]
Saving to: ‘blp25_hatespeech_subtask_1C_train.tsv’

blp25_hatespeech_su 100%[===================>]   8.28M  --.-KB/s    in 0.08s   

2025-09-12 19:26:37 (109 MB/s) - ‘blp25_hatespeech_subtask_1C_train.tsv’ saved [8686966/8686966]

--2025-09-12 19:26:37--  https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1C/blp25_hatespeech_subtask_1C_dev.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubu

In [2]:
import random
random.seed(123)


In [3]:
!pip uninstall wandb --y

Found existing installation: wandb 0.21.3
Uninstalling wandb-0.21.3:
  Successfully uninstalled wandb-0.21.3


In [4]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.1 MB/s eta 0:00:00


In [5]:
import numpy as np
import os
from datetime import datetime

import datasets
import evaluate

import torch
import torch.nn as nn

from transformers import Trainer, TrainerCallback, TrainingArguments, EarlyStoppingCallback
from transformers import AutoTokenizer, AutoConfig, AutoModel, PreTrainedModel

In [6]:
MAX_LENGTH = 512
NUM_TRAIN_EPOCHS = 10

timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

RESULTS_DIRECTORY = './results/experiment_{}'.format(timestamp)
LOGGING_DIRECTORY = './logs/experiement_{}'.format(timestamp)

RESULTS_DIRECTORY, LOGGING_DIRECTORY

('./results/experiment_2025-09-12_19-27-34',
 './logs/experiement_2025-09-12_19-27-34')

In [7]:
# Label sets and mappings for the new dataset (Subtask 1C)

HATE_TYPE_LABELS = [
    'None', 'Abusive', 'Profane', 'Religious Hate', 'Political Hate', 'Sexism'
]
HATE_SEVERITY_LABELS = [
    'Little to None', 'Mild', 'Severe'
]
TO_WHOM_LABELS = [
    'None', 'Individual', 'Society', 'Community', 'Organization'
]

HATE_TYPE_LABELS_TO_IDX = {label: idx for idx, label in enumerate(HATE_TYPE_LABELS)}
HATE_SEVERITY_LABELS_TO_IDX = {label: idx for idx, label in enumerate(HATE_SEVERITY_LABELS)}
TO_WHOM_LABELS_TO_IDX = {label: idx for idx, label in enumerate(TO_WHOM_LABELS)}

IDX_TO_HATE_TYPE_LABELS = {v: k for k, v in HATE_TYPE_LABELS_TO_IDX.items()}
IDX_TO_HATE_SEVERITY_LABELS = {v: k for k, v in HATE_SEVERITY_LABELS_TO_IDX.items()}
IDX_TO_TO_WHOM_LABELS = {v: k for k, v in TO_WHOM_LABELS_TO_IDX.items()}

MAX_SEQUENCE_LENGTH = 512
PRETRAINED_MODEL_NAME = "microsoft/mdeberta-v3-base"
NUM_TRAIN_EPOCHS = 10

In [8]:
# tokenize function (unchanged)
def tokenize(example, tokenizer):
    return tokenizer(
        example['text'],
        padding=True,
        truncation=True,
        max_length=MAX_SEQUENCE_LENGTH
    )

# SAFE helpers
def _norm(x):
    if isinstance(x, str):
        return x.strip()
    return x

def _safe_map(val, mapping, default_key):
    v = _norm(val)
    # Treat None/NaN/missing as default
    if v is None:
        return mapping[default_key]
    try:
        # np.isnan only for floats
        import numpy as _np
        if isinstance(v, float) and _np.isnan(v):
            return mapping[default_key]
    except Exception:
        pass
    return mapping.get(v, mapping[default_key])

# label encoding with safe defaults
def label_encoder(example):
    example['hate_type_label'] = _safe_map(example.get('hate_type'), HATE_TYPE_LABELS_TO_IDX, 'None')
    example['hate_severity_label'] = _safe_map(example.get('hate_severity'), HATE_SEVERITY_LABELS_TO_IDX, 'Little to None')
    example['to_whom_label'] = _safe_map(example.get('to_whom'), TO_WHOM_LABELS_TO_IDX, 'None')
    return example

In [9]:
CONFIG = AutoConfig.from_pretrained(PRETRAINED_MODEL_NAME)
TOKENIZER = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME, use_fast=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [10]:
# loading data (UPDATED to use your TSVs instead of load_from_disk)
import pandas as pd
from datasets import Dataset, DatasetDict

train_path = "/content/blp25_hatespeech_subtask_1C_train.tsv"
dev_path   = "/content/blp25_hatespeech_subtask_1C_dev.tsv"

train_df = pd.read_csv(train_path, sep="\t")
dev_df   = pd.read_csv(dev_path, sep="\t")

# Ensure required columns are present
required_cols = ["text", "hate_type", "hate_severity", "to_whom"]
if "id" in train_df.columns:
    train_df = train_df[["id"] + required_cols]
else:
    train_df = train_df[required_cols]

if "id" in dev_df.columns:
    dev_df = dev_df[["id"] + required_cols]
else:
    dev_df = dev_df[required_cols]

# Build HF datasets; match original notebook’s structure
data = DatasetDict(
    {
        "train": Dataset.from_pandas(train_df, preserve_index=False),
        "test":  Dataset.from_pandas(dev_df,   preserve_index=False),
    }
)
data

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'hate_type', 'hate_severity', 'to_whom'],
        num_rows: 35522
    })
    test: Dataset({
        features: ['id', 'text', 'hate_type', 'hate_severity', 'to_whom'],
        num_rows: 2512
    })
})

In [11]:
# tokenize + label encode with the updated functions from Cell 4
data = data.map(lambda x: tokenize(x, tokenizer=TOKENIZER))
data = data.map(lambda x: label_encoder(x))
data

Map:   0%|          | 0/35522 [00:00<?, ? examples/s]

Map:   0%|          | 0/2512 [00:00<?, ? examples/s]

Map:   0%|          | 0/35522 [00:00<?, ? examples/s]

Map:   0%|          | 0/2512 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'hate_type', 'hate_severity', 'to_whom', 'input_ids', 'token_type_ids', 'attention_mask', 'hate_type_label', 'hate_severity_label', 'to_whom_label'],
        num_rows: 35522
    })
    test: Dataset({
        features: ['id', 'text', 'hate_type', 'hate_severity', 'to_whom', 'input_ids', 'token_type_ids', 'attention_mask', 'hate_type_label', 'hate_severity_label', 'to_whom_label'],
        num_rows: 2512
    })
})

In [12]:
# Defining the metrics (three heads)
# We will use weighted F1 for hate_type, hate_severity, and to_whom
HATE_TYPE_METRIC = evaluate.load("f1")
HATE_SEVERITY_METRIC = evaluate.load("f1")
TO_WHOM_METRIC = evaluate.load("f1")

def compute_metrics(eval_pred):
    # Trainer passes (predictions, labels)
    all_logits, all_labels = eval_pred

    # Unpack logits from the three heads (order must match model outputs)
    hate_type_logits, hate_severity_logits, to_whom_logits = all_logits

    # Unpack labels in the same order as label_names in TrainingArguments
    hate_type_labels, hate_severity_labels, to_whom_labels = all_labels

    # Argmax to get predictions
    hate_type_predictions = np.argmax(hate_type_logits, axis=-1)
    hate_severity_predictions = np.argmax(hate_severity_logits, axis=-1)
    to_whom_predictions = np.argmax(to_whom_logits, axis=-1)

    # Compute weighted F1 for each head
    hate_type_metrics = HATE_TYPE_METRIC.compute(
        predictions=hate_type_predictions,
        references=hate_type_labels,
        average='weighted'
    )
    hate_severity_metrics = HATE_SEVERITY_METRIC.compute(
        predictions=hate_severity_predictions,
        references=hate_severity_labels,
        average='weighted'
    )
    to_whom_metrics = TO_WHOM_METRIC.compute(
        predictions=to_whom_predictions,
        references=to_whom_labels,
        average='weighted'
    )

    return {
        'f1_hate_type': hate_type_metrics['f1'],
        'f1_hate_severity': hate_severity_metrics['f1'],
        'f1_to_whom': to_whom_metrics['f1'],
    }

In [13]:
# model definition
class MultiTaskSentencePrediction(PreTrainedModel):
    config_class = AutoConfig

    def __init__(self, config, num_hate_type_labels, num_hate_severity_labels, num_to_whom_labels):
        super().__init__(config)
        self.num_hate_type_labels = num_hate_type_labels
        self.num_hate_severity_labels = num_hate_severity_labels
        self.num_to_whom_labels = num_to_whom_labels

        # Auto backbone (IndicBERTv2 বা যেকোনো compatible checkpoint)
        self.bert = AutoModel.from_config(config)

        self.hate_type_classifier = nn.Linear(config.hidden_size, num_hate_type_labels)
        self.hate_severity_classifier = nn.Linear(config.hidden_size, num_hate_severity_labels)
        self.to_whom_classifier = nn.Linear(config.hidden_size, num_to_whom_labels)

        classifier_dropout = getattr(config, "classifier_dropout", None)
        if classifier_dropout is None:
            classifier_dropout = getattr(config, "hidden_dropout_prob", 0.1)
        self.dropout = nn.Dropout(classifier_dropout)

        self.post_init()  # modern replacement for init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
        hate_type_label=None,
        hate_severity_label=None,
        to_whom_label=None
    ):
        # simplest fix — drop head_mask
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=True,
        )

        # pooler_output না থাকলে [CLS] টোকেন ব্যবহার
        pooled_output = getattr(outputs, "pooler_output", None)
        if pooled_output is None:
            pooled_output = outputs.last_hidden_state[:, 0]

        pooled_output = self.dropout(pooled_output)

        logits_hate_type = self.hate_type_classifier(pooled_output)
        logits_hate_severity = self.hate_severity_classifier(pooled_output)
        logits_to_whom = self.to_whom_classifier(pooled_output)

        loss = None
        if (hate_type_label is not None) and (hate_severity_label is not None) and (to_whom_label is not None):
            loss_fct = nn.CrossEntropyLoss()
            loss = (
                loss_fct(logits_hate_type.view(-1, self.num_hate_type_labels), hate_type_label.view(-1)) +
                loss_fct(logits_hate_severity.view(-1, self.num_hate_severity_labels), hate_severity_label.view(-1)) +
                loss_fct(logits_to_whom.view(-1, self.num_to_whom_labels), to_whom_label.view(-1))
            )

        # Trainer compatibility
        if loss is not None:
            return (loss, (logits_hate_type, logits_hate_severity, logits_to_whom))
        else:
            return (logits_hate_type, logits_hate_severity, logits_to_whom)

In [14]:

# Three-head labels
label_names = ['hate_type_label', 'hate_severity_label', 'to_whom_label']

load_best_model_at_end = True
metric_for_best_model = 'eval_f1_hate_type'  # still used when you call trainer.evaluate()
greater_is_better = True
label_smoothing_factor = 0
report_to = 'tensorboard'
gradient_checkpointing = False

In [15]:
def data_collator(batch, padding_token_id=TOKENIZER.pad_token_id):
    # Truncate first
    input_ids = [item["input_ids"][:MAX_SEQUENCE_LENGTH] for item in batch]
    attention_masks = [item["attention_mask"][:MAX_SEQUENCE_LENGTH] for item in batch]

    # Labels (already int-encoded)
    hate_type_label = [item["hate_type_label"] for item in batch]
    hate_severity_label = [item["hate_severity_label"] for item in batch]
    to_whom_label = [item["to_whom_label"] for item in batch]

    # Dynamic pad to max len in batch
    max_len = max(len(ids) for ids in input_ids)
    input_ids = torch.tensor([ids + [padding_token_id] * (max_len - len(ids)) for ids in input_ids])
    attention_masks = torch.tensor([m + [0] * (max_len - len(m)) for m in attention_masks])

    hate_type_label = torch.tensor(hate_type_label)
    hate_severity_label = torch.tensor(hate_severity_label)
    to_whom_label = torch.tensor(to_whom_label)

    return {
        "input_ids": input_ids,
        "attention_mask": attention_masks,
        "hate_type_label": hate_type_label,
        "hate_severity_label": hate_severity_label,
        "to_whom_label": to_whom_label
    }


In [16]:
# Setup training arguments (no evaluation during training)
training_args = TrainingArguments(
    # === core training ===
    learning_rate=2e-5,
    num_train_epochs=4,                 # same as before
    per_device_train_batch_size=16,     # unchanged
    per_device_eval_batch_size=16,      # unchanged
    gradient_accumulation_steps=1,      # unchanged

    # === saving & checkpointing ===
    save_strategy="epoch",              # save checkpoints each epoch
    save_total_limit=2,                 # keep last 2 checkpoints

    # === optimization recipe ===
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,                  # unchanged; works fine for IndicBERT v2
    weight_decay=0.01,

    # === misc / perf ===
    remove_unused_columns=False,
    report_to=None,                     # disable TensorBoard/W&B logging
    seed=42,
    fp16=True,                          # if GPU supports (Ampere+)
)

# Remove EarlyStopping because it relies on eval during training
# early_stop_callback = EarlyStoppingCallback(3)

In [17]:
# Initialize the model
model = MultiTaskSentencePrediction.from_pretrained(
    PRETRAINED_MODEL_NAME,
    config=CONFIG,
    num_hate_type_labels=len(HATE_TYPE_LABELS),
    num_hate_severity_labels=len(HATE_SEVERITY_LABELS),
    num_to_whom_labels=len(TO_WHOM_LABELS),
)

pytorch_model.bin:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of MultiTaskSentencePrediction were not initialized from the model checkpoint at microsoft/mdeberta-v3-base and are newly initialized: ['bert.embeddings.LayerNorm.bias', 'bert.embeddings.LayerNorm.weight', 'bert.embeddings.word_embeddings.weight', 'bert.encoder.LayerNorm.bias', 'bert.encoder.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.dense.bias', 'bert.encoder.layer.0.attention.output.dense.weight', 'bert.encoder.layer.0.attention.self.key_proj.bias', 'bert.encoder.layer.0.attention.self.key_proj.weight', 'bert.encoder.layer.0.attention.self.query_proj.bias', 'bert.encoder.layer.0.attention.self.query_proj.weight', 'bert.encoder.layer.0.attention.self.value_proj.bias', 'bert.encoder.layer.0.attention.self.value_proj.weight', 'bert.encoder.layer.0.intermediate.dense.bias', 'bert.encoder.layer.0.intermediate.dense.weight', 'bert.encoder.layer.0.out

In [18]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data['train'],
    eval_dataset=data['test'],
    tokenizer=TOKENIZER,
    compute_metrics=compute_metrics,
    data_collator=data_collator
    # callbacks=[early_stop_callback]  # removed
)


/tmp/ipython-input-3144340026.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [19]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.


Step,Training Loss
500,3.303100
1000,2.974800
1500,2.803800
2000,2.675100
2500,2.548300
3000,2.450700
3500,2.512600
4000,2.456100
4500,2.420900
5000,2.265700


TrainOutput(global_step=8884, training_loss=2.454689599543421, metrics={'train_runtime': 2442.2646, 'train_samples_per_second': 58.179, 'train_steps_per_second': 3.638, 'total_flos': 8826991560041160.0, 'train_loss': 2.454689599543421, 'epoch': 4.0})

'eval_loss': 2.2633869647979736, 'eval_f1_hate_type': 0.7219821381799193, 'eval_f1_hate_severity': 0.7148860272374146, 'eval_f1_to_whom': 0.7147589711289586, 'eval_runtime': 3.5021, 'eval_samples_per_second': 717.285, 'eval_steps_per_second': 44.83, 'epoch': 4.0}

In [20]:
eval_metrics = trainer.evaluate()
print(eval_metrics)

{'eval_loss': 2.3737871646881104, 'eval_f1_hate_type': 0.6588568899038006, 'eval_f1_hate_severity': 0.6777109646826412, 'eval_f1_to_whom': 0.6549804784344088, 'eval_runtime': 9.2686, 'eval_samples_per_second': 271.022, 'eval_steps_per_second': 16.939, 'epoch': 4.0}


In [21]:
import pandas as pd
from datasets import Dataset

test_path = "/content/blp25_hatespeech_subtask_1C_test.tsv"
test_df = pd.read_csv(test_path, sep="\t")

print("Test shape:", test_df.shape)
print(test_df.head())

# HuggingFace Dataset এ কনভার্ট
test_dataset = Dataset.from_pandas(test_df)

# Tokenize
test_dataset = test_dataset.map(lambda x: TOKENIZER(
    x["text"],
    padding="max_length",
    truncation=True,
    max_length=MAX_SEQUENCE_LENGTH
), batched=True)

# Dummy labels যোগ করা collator satisfy করার জন্য
test_dataset = test_dataset.map(lambda x: {
    "hate_type_label": 0,
    "hate_severity_label": 0,
    "to_whom_label": 0
})

# Collator যেন list পায়, তাই format None
test_dataset.set_format(
    type=None,
    columns=["input_ids", "attention_mask",
             "hate_type_label", "hate_severity_label", "to_whom_label"]
)

Test shape: (10200, 2)
       id                                               text
0   12764                        ইজরায়েলের বিচার হওয়া উচিৎ
1  202933                             শামীম ওসামা বিন হাসিনা
2  165894  হেন কাপ পুলিশের মারে অন্যরা তাহলে পলিশের কি হব...
3  124999                আল্লাহ্ এসব জানোয়ারদের শেষ করে দাও
4  535301                          ইহুদির বাচ্চা ইহুদী ই হবে


Map:   0%|          | 0/10200 [00:00<?, ? examples/s]

Map:   0%|          | 0/10200 [00:00<?, ? examples/s]

In [22]:
# Runtime model দিয়েই predict
predictions = trainer.predict(test_dataset)

# তিনটা head এর logits
logits_hate_type, logits_hate_severity, logits_to_whom = predictions.predictions

import numpy as np
hate_type_preds = np.argmax(logits_hate_type, axis=-1)
hate_severity_preds = np.argmax(logits_hate_severity, axis=-1)
to_whom_preds = np.argmax(logits_to_whom, axis=-1)

In [23]:
pred_df = pd.DataFrame({
    "id": test_df["id"],
    "hate_type": [IDX_TO_HATE_TYPE_LABELS[int(i)] for i in hate_type_preds],
    "hate_severity": [IDX_TO_HATE_SEVERITY_LABELS[int(i)] for i in hate_severity_preds],
    "to_whom": [IDX_TO_TO_WHOM_LABELS[int(i)] for i in to_whom_preds],
    "model": PRETRAINED_MODEL_NAME
})

# Save as TSV (tab separated)
pred_df.to_csv("subtask_1C.tsv", sep="\t", index=False)

print("✅ subtask_1C.tsv saved with columns: id, hate_type, hate_severity, to_whom, model")
pred_df.head()

✅ subtask_1C.tsv saved with columns: id, hate_type, hate_severity, to_whom, model


,id,hate_type,hate_severity,to_whom,model
0,12764,None,Little to None,None,microsoft/mdeberta-v3-base
1,202933,None,Little to None,None,microsoft/mdeberta-v3-base
2,165894,None,Little to None,None,microsoft/mdeberta-v3-base
3,124999,Profane,Severe,Community,microsoft/mdeberta-v3-base
4,535301,Religious Hate,Severe,Individual,microsoft/mdeberta-v3-base


In [24]:
!zip subtask_1C.zip subtask_1C.tsv

  adding: subtask_1C.tsv (deflated 91%)
